### init kennel

In [2]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs 
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.8.0"
#r "nuget: Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.8.0"
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.PromptTemplates.Handlebars;

//Create Kernel builder
var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();


  // Create a Semantic Kernel template for chat
        var chat = kernel.CreateFunctionFromPrompt(
            @"{{$history}}
            User: {{$request}}
            Assistant: ");

        // Create choices
        List<string> choices = ["ContinueConversation", "EndConversation"];

        // Create few-shot examples
        List<ChatHistory> fewShotExamples =
        [
            [
                new ChatMessageContent(AuthorRole.User, "Can you send a very quick approval to the marketing team?"),
                new ChatMessageContent(AuthorRole.System, "Intent:"),
                new ChatMessageContent(AuthorRole.Assistant, "ContinueConversation")
            ],
            [
                new ChatMessageContent(AuthorRole.User, "Thanks, I'm done for now"),
                new ChatMessageContent(AuthorRole.System, "Intent:"),
                new ChatMessageContent(AuthorRole.Assistant, "EndConversation")
            ]
        ];


        // Create handlebars template for intent
        var getIntent = kernel.CreateFunctionFromPrompt(
            new()
            {
                Template = """
                           <message role="system">Instructions: What is the intent of this request?
                           Do not explain the reasoning, just reply back with the intent. If you are unsure, reply with {{choices[0]}}.
                           Choices: {{choices}}.</message>

                           {{#each fewShotExamples}}
                               {{#each this}}
                                   <message role="{{role}}">{{content}}</message>
                               {{/each}}
                           {{/each}}
                           {{ConversationSummaryPlugin-SummarizeConversation history}}
                           <message role="user">{{request}}</message>
                           <message role="system">Intent:</message>
                           """,
                TemplateFormat = "handlebars"
            },
            new HandlebarsPromptTemplateFactory()
        );

        ChatHistory history = [];
   // Start the chat loop
        while (true)
        {
            // Get user input
           Console.Write("User > ");
            var request =Console.ReadLine();

            // Invoke prompt
            var intent = await kernel.InvokeAsync(
                getIntent,
                new()
                {
                    { "request", request },
                    { "choices", choices },
                    { "history", history },
                    { "fewShotExamples", fewShotExamples }
                }
            );

            // End the chat if the intent is "Stop"
            if (intent.ToString() == "EndConversation")
            {
                break;
            }

            // Get chat response
            var chatResult = kernel.InvokeStreamingAsync<StreamingChatMessageContent>(
                chat,
                new()
                {
                    { "request", request },
                    { "history", string.Join("\n", history.Select(x => x.Role + ": " + x.Content)) }
                }
            );

            // Stream the response
            string message = "";
            await foreach (var chunk in chatResult)
            {
                if (chunk.Role.HasValue)
                {
                    Console.Write(chunk.Role + " > ");
                }

                message += chunk;
                Console.Write(chunk);
            }
            Console.WriteLine();

            // Append to history
            history.AddUserMessage(request!);
            history.AddAssistantMessage(message);
        }

Installed Packages Microsoft.SemanticKernel, 1.8.0 Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.8.0

User > 

Error: HandlebarsDotNet.HandlebarsRuntimeException: Template references a helper that cannot be resolved. Helper 'ConversationSummaryPlugin-SummarizeConversation'
   at HandlebarsDotNet.Helpers.MissingHelperDescriptor.Invoke(HelperOptions& options, Context& context, Arguments& arguments)
   at HandlebarsDotNet.Helpers.MissingHelperDescriptor.HandlebarsDotNet.Helpers.IHelperDescriptor<HandlebarsDotNet.HelperOptions>.Invoke(HelperOptions& options, Context& context, Arguments& arguments)
   at HandlebarsDotNet.Helpers.LateBindHelperDescriptor.Invoke(HelperOptions& options, Context& context, Arguments& arguments)
   at HandlebarsDotNet.Helpers.LateBindHelperDescriptor.Invoke(EncodedTextWriter& output, HelperOptions& options, Context& context, Arguments& arguments)
   at HandlebarsDotNet.Helpers.LateBindHelperDescriptor.HandlebarsDotNet.Helpers.IHelperDescriptor<HandlebarsDotNet.HelperOptions>.Invoke(EncodedTextWriter& output, HelperOptions& options, Context& context, Arguments& arguments)
